# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [ ]:
#Adding libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
#doing the same feature eng as before
spaceship.dropna(inplace=True)

spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x.split('/')[0])
spaceship.drop(columns=['PassengerId','Name'], inplace=True)

spaceship = pd.get_dummies(spaceship, drop_first=True)

# split
X = spaceship.drop(columns='Transported')
y = spaceship['Transported']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# now we scale the features this time
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [5]:
from sklearn.neighbors import KNeighborsClassifier

# knn was our best model so lets use that
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

- Evaluate your model

In [6]:
print(f"train acc: {knn.score(X_train, y_train)}")
print(f"test acc: {knn.score(X_test, y_test)}")
# ok so this is the basline before tuning

train acc: 0.825889477668433
test acc: 0.7874432677760969


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [7]:
from sklearn.model_selection import GridSearchCV

# params to try
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

- Run Grid Search

In [8]:
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(f"best params: {grid_search.best_params_}")
print(f"best cv score: {grid_search.best_score_}")

best params: {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'uniform'}
best cv score: 0.7746038315988647


- Evaluate your model

In [9]:
#using the best model from gridsearch
best_knn = grid_search.best_estimator_

print(f"train acc: {best_knn.score(X_train, y_train)}")
print(f"test acc: {best_knn.score(X_test, y_test)}")

#should be better thn the defualt knn had before
# the gridsearch tryed all the combintions and picked the best one

train acc: 0.8067751703255109
test acc: 0.7965204236006052
